In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords
from collections import Counter
import nltk
from sklearn.datasets import fetch_20newsgroups

In [2]:
%%time
newsgroups_train = fetch_20newsgroups(subset='train')

CPU times: user 240 ms, sys: 67.7 ms, total: 307 ms
Wall time: 318 ms


In [53]:
type(newsgroups_train)

sklearn.utils.Bunch

In [3]:
%%time
from pprint import pprint
# view the dictionary keys
pprint(list(newsgroups_train))

['data', 'filenames', 'target_names', 'target', 'DESCR']
CPU times: user 648 µs, sys: 789 µs, total: 1.44 ms
Wall time: 822 µs


In [ ]:
def sklearn_to_df(sklearn_dataset):
    df = pd.DataFrame(sklearn_dataset.data, columns=sklearn_dataset.feature_names)
    df['target'] = pd.Series(sklearn_dataset.target)
    return df

df_boston = sklearn_to_df(datasets.load_boston())

In [ ]:
data1 = pd.DataFrame(data= np.c_[iris['data'], iris['target']],
                 columns= list(iris['feature_names']) + ['target'])

In [102]:
%%time
# np.c_ is the numpy concatenate function
# which is used to concat iris['data'] and iris['target'] arrays 
# for pandas column argument: concat iris['feature_names'] list
# and string list (in this case one string); you can make this anything you'd like..  
# the original dataset would probably call this ['Species']
df_news = pd.DataFrame(data=[newsgroups_train['data'], newsgroups_train['filenames'],
                       newsgroups_train['target_names'], newsgroups_train['target'],
                       newsgroups_train['DESCR']],
                     columns= list(newsgroups_train['data'], newsgroups_train['filenames'],
                       newsgroups_train['target_names'], newsgroups_train['target'],
                       newsgroups_train['DESCR']))

TypeError: list() takes at most 1 argument (5 given)

In [71]:
newsgroups_train = fetch_20newsgroups(subset='train')
# convert the sklearn.utils.Bunch into a DataFrame
def sklearn_to_df(sklearn_dataset):
    df = pd.DataFrame(newsgroups_train, columns=newsgroups_train.data, 
                      newsgroups_train.filenames, newsgroups_train.target_names,
                      newsgroups_train.target, newsgroups_train.DESCR)
    df['data'] = pd.Series(newsgroups_train.data)
    #df['filesnames'] = pd.Series(newsgroups_train.filenames)
    #df['target_names'] = pd.Series(newsgroups_train.target_names)
    #df['target'] = pd.Series(newsgroups_train.target)
    #df['DESCR'] = pd.Series(newsgroups_train.DESCR)
    return df

df_news = sklearn_to_df(fetch_20newsgroups(subset='train'))
    
  

SyntaxError: positional argument follows keyword argument (<ipython-input-71-3d3e486aff20>, line 5)

In [62]:
newsgroups_train = fetch_20newsgroups(subset='train')
# convert the sklearn.utils.Bunch into a DataFrame
def sklearn_to_df(sklearn_dataset):
    df = pd.DataFrame(newsgroups_train, columns=newsgroups_train[['data', 
                                            'filenames', 'target_names', 'target', 'DESCR']])
    df['data'] = pd.Series(newsgroups_train.data)
    df['filesnames'] = pd.Series(newsgroups_train.filenames)
    df['target_names'] = pd.Series(newsgroups_train.target_names)
    df['target'] = pd.Series(newsgroups_train.target)
    df['DESCR'] = pd.Series(newsgroups_train.DESCR)
    return df

df_news = sklearn_to_df(fetch_20newsgroups(subset='train'))
    
  

TypeError: unhashable type: 'list'

In [54]:
  
newsgroups_train = fetch_20newsgroups(subset='train')
df_newsgroups_train = pd.DataFrame(newsgroups_train.newsgroups_train,
                                   columns=[newsgroups_train.data,
                                            newsgroups_train.filenames,
                                            newsgroups_train.target_names,
                                            newsgroups_train.target,
                                            newsgroups_train.DESCR])
df_newsgroups_train.head()

AttributeError: newsgroups_train

In [4]:
%%time
# the different article categories
pprint(list(newsgroups_train.target_names))

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']
CPU times: user 3.63 ms, sys: 5.24 ms, total: 8.87 ms
Wall time: 7.17 ms


In [5]:
%%time
# understanding the length and shape of the file
print(newsgroups_train.filenames.shape)
print(newsgroups_train.target.shape)
print(newsgroups_train.target[:10])


(11314,)
(11314,)
[ 7  4  4  1 14 16 13  3  2  4]
CPU times: user 911 µs, sys: 1.07 ms, total: 1.98 ms
Wall time: 977 µs


In [ ]:
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text

In [44]:
%%time
#from collections import Counter
from sklearn.datasets import fetch_20newsgroups
import re

# define text cleaner function for the dataset
def text_cleaner(line:str):
    '''define the function for a text cleaner to run the news through'''
        # replaces new line with a space
    line = re.sub('\n', ' ', line)
        # removes characters outside of a-z w/ a space
    line = re.sub('[^a-z ]', ' ', line)
        # removes things within a [] bracket and replaces w/ a space
    line = re.sub("[\[].*?[\]]", ' ', line)
        # removes -- and replaces w/ a space
    line = re.sub(r'--', ' ', line)
        # splits sentences and makes it a list 
    line = ' '.join(line.split())
    return line
        #tokens = line.split(' ')
        #tokens = [token for token in tokens if len(token) > 0]
        #tokens_len = [len(token) for token in tokens if len(token) > 0]
        #corpus.extend(tokens)
    
   # corpus = Counter(corpus)

  #  corpus_dir = '../../data/'
  #  corpus_file_name = 'spell_check_dictionary.txt'

CPU times: user 19 µs, sys: 1 µs, total: 20 µs
Wall time: 26 µs


In [39]:
%%time
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text:str):
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in line
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]

CPU times: user 7 µs, sys: 1 µs, total: 8 µs
Wall time: 11 µs


In [40]:
# saving for pythonic example of a good for loop
#tokens = [token for token in tokens if len(token) > 0]

In [48]:
%%time
from sklearn.feature_extraction.text import TfidfVectorizer
## create categories 
categories = ['misc.forsale', 'talk.politics.mideast', 
              'sci.space', 'rec.sport.baseball']

# create a new variable for the reduced dataset
# to 4 categories for faster NLP
newsgroups_train_reduced = fetch_20newsgroups(subset='train',
                        categories=categories, shuffle=True, random_state=9)

CPU times: user 247 ms, sys: 61.9 ms, total: 309 ms
Wall time: 324 ms


In [49]:
type(newsgroups_train_reduced)

sklearn.utils.Bunch

In [51]:
# clean the dataset
news = newsgroups_train_reduced.values

In [52]:
type(news)

builtin_function_or_method

In [50]:
news = text_cleaner(newsgroups_train_reduced)

TypeError: expected string or bytes-like object

In [9]:
%%time
import spacy
nlp = spacy.load('en')
nlp.max_length = 23065807
news_doc = nlp(" ".join(newsgroups_train_reduced["data"]))

CPU times: user 2min 37s, sys: 1min 21s, total: 3min 58s
Wall time: 5min 18s


In [13]:
type(nlp)

spacy.lang.en.English

In [10]:
%%time
news_sents = [[sent, "data"] for sent in news_doc.sents]

# Create one data frame.
sentences = pd.DataFrame(news_sents)
sentences.head()

CPU times: user 243 ms, sys: 329 ms, total: 572 ms
Wall time: 834 ms
